## Imports

In [86]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [87]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Oct. 8th Oct. 15th


In [88]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [89]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [90]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [91]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [92]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,October-10-2024,Condo/Co-op,3507 Oaks Way #808,Pompano Beach,FL,33069.0,215000.0,2.0,2.0,Palm Aire,1370.0,NaN,1981.0,NaN,157.0,749.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/3507-O...,Beaches MLS,F10440759,N,Y,26.226257,-80.176679


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [93]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [94]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
36,PAST SALE,2024-10-11,Condo/Co-op,2221 Cypress Island Dr #508,Pompano Beach,FL,33069.0,245000.0,2.0,2.0,Cypress Bend,1330.0,NaN,1987.0,NaN,184.0,678.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2221-C...,Beaches MLS,F10448194,N,Y,26.216001,-80.151014
28,PAST SALE,2024-10-11,Condo/Co-op,2900 NE 30th St Unit I-7,Fort Lauderdale,FL,33306.0,290000.0,1.0,1.0,Lauderdale Tower Condo,730.0,NaN,1964.0,NaN,397.0,812.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2900...,Beaches MLS,F10455672,N,Y,26.166063,-80.104921
32,PAST SALE,2024-10-11,Condo/Co-op,1700 SE 15th St #111,Fort Lauderdale,FL,33316.0,500000.0,2.0,2.0,Caravelle,1200.0,NaN,1965.0,NaN,417.0,676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1700...,Beaches MLS,F10451896,N,Y,26.102984,-80.123814
23,PAST SALE,2024-10-11,Condo/Co-op,12620 Vista Isles Dr #1024,Sunrise,FL,33325.0,200000.0,1.0,1.0,ISLES AT LAGO MAR CONDO,743.0,NaN,1991.0,NaN,269.0,305.0,Sold,NaN,NaN,https://www.redfin.com/FL/Plantation/12620-Vis...,MARMLS,A11618571,N,Y,26.130308,-80.317441
21,PAST SALE,2024-10-11,Condo/Co-op,2000 Atlantic Shores Blvd #503,Hallandale,FL,33009.0,210000.0,2.0,1.5,Desoto Park Condominiums,940.0,NaN,1974.0,NaN,223.0,614.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/200...,Beaches MLS,F10457541,N,Y,25.995037,-80.128224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,PAST SALE,2024-10-07,Condo/Co-op,301 N Pine Island Rd #159,Plantation,FL,33324.0,235000.0,2.0,2.0,RAINTREE FOREST CONDO,1275.0,NaN,1975.0,NaN,184.0,696.0,Sold,NaN,NaN,https://www.redfin.com/FL/Plantation/301-N-Pin...,MARMLS,A11644538,N,Y,26.123358,-80.264762
43,PAST SALE,2024-10-07,Condo/Co-op,2586 Centergate Dr #206,Miramar,FL,33025.0,310000.0,2.0,2.0,AVENTINE AT MIRAMAR CONDO,1072.0,NaN,2003.0,NaN,289.0,324.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miramar/2586-Centerg...,MARMLS,A11604960,N,Y,25.985162,-80.310789
42,PAST SALE,2024-10-07,Condo/Co-op,2251 NE 66th St #1623,Fort Lauderdale,FL,33308.0,168000.0,2.0,2.0,IMPERIAL POINT COLONNADES,1030.0,NaN,1971.0,NaN,163.0,678.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2251...,MARMLS,A11587613,N,Y,26.209241,-80.110984
41,PAST SALE,2024-10-07,Condo/Co-op,1630 N Ocean Blvd #712,Pompano Beach,FL,33062.0,500000.0,2.0,2.0,Tiffany Gardens West,975.0,NaN,1972.0,NaN,513.0,565.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1630-N...,Beaches MLS,F10443703,N,Y,26.252622,-80.085693


In [95]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [96]:
df_top_ten = df.head(10)

## Collect Agent Information

In [97]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [98]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [99]:
response_df = pd.DataFrame(response_list)

In [100]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [101]:
df_top_ten = merged_df

## Current Week's Values

In [102]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

70


In [103]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$294,649


In [104]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$253


In [105]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$20,625,400


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [106]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
142
Input Previous Week Condo Average Sales Price:
388150
Input Previous Week Condo Average PSF:
305
Input Previous Week Condo Sales total (ex: 198_000_000)
55_100_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [107]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [108]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [109]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [110]:
pd.set_option('display.max_columns',None)

In [111]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-07,Condo/Co-op,615 Bayshore Dr #303,Fort Lauderdale,FL,33304.0,1200000.0,3.0,3.0,LA Cascade Condo,2365.0,NaN,2003.0,NaN,507.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Beaches MLS,RX-11016889,N,Y,26.131947,-80.109184,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Lawrence Mastropieri,Mastropieri Group LLC,Joy Mastropieri,Mastropieri Group LLC,Robin Johannan,Fortune Christie's International Real Estate,None,None,1,orange


In [112]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [113]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-07,Condo/Co-op,615 Bayshore Dr #303,Fort Lauderdale,FL,33304.0,1200000.0,3.0,3.0,LA Cascade Condo,2365.0,NaN,2003.0,NaN,507.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Beaches MLS,RX-11016889,N,Y,26.131947,-80.109184,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Lawrence Mastropieri,Mastropieri Group LLC,Joy Mastropieri,Mastropieri Group LLC,Robin Johannan,Fortune Christie's International Real Estate,None,None,1,orange
1,PAST SALE,2024-10-09,Condo/Co-op,401 N Birch Rd #1102,Fort Lauderdale,FL,33304.0,1200000.0,2.0,2.0,Tiffany House Residences,1286.0,NaN,2018.0,NaN,933.0,1888.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Beaches MLS,F10455561,N,Y,26.128334,-80.106186,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Dayana Susterman,"Compass Florida, LLC",Joshua Dotoli,"Compass Florida, LLC",Deborah Haines,Trophy Properties South Florida Inc,None,None,2,blue
2,PAST SALE,2024-10-11,Condo/Co-op,615 Bayshore Dr #401,Fort Lauderdale,FL,33304.0,1050000.0,2.0,2.5,LA Cascade Condo,2274.0,NaN,2003.0,NaN,462.0,1650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Beaches MLS,RX-10942553,N,Y,26.131947,-80.109184,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Brian Dratt,Partnership Realty Inc.,None,None,Brian Dratt,Partnership Realty Inc.,None,None,3,blue
3,PAST SALE,2024-10-07,Condo/Co-op,111 N Pompano Beach Blvd #1411,Pompano Beach,FL,33062.0,715000.0,2.0,2.0,Sea Monarch,1455.0,NaN,1970.0,NaN,491.0,1020.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/111-N-...,Beaches MLS,F10459447,N,Y,26.234032,-80.090070,https://www.redfin.com/FL/Pompano-Beach/111-N-...,Chad Bishop,One Sotheby's Int'l Realty,Michael Clouston,One Sotheby's Int'l Realty,Robin Michel Gervais,Best of Luxury Realty Corp.,None,None,4,blue
4,PAST SALE,2024-10-10,Condo/Co-op,110 N Federal Hwy #805,Fort Lauderdale,FL,33301.0,520000.0,2.0,2.0,The Waverly At Las Olas,1350.0,1350.0,2004.0,NaN,385.0,1389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/110-...,Beaches MLS,F10450734,N,Y,26.123305,-80.136965,https://www.redfin.com/FL/Fort-Lauderdale/110-...,Adam Curtis,LoKation,None,None,Adam Curtis,LoKation,None,None,5,blue
5,PAST SALE,2024-10-07,Condo/Co-op,1630 N Ocean Blvd #712,Pompano Beach,FL,33062.0,500000.0,2.0,2.0,Tiffany Gardens West,975.0,NaN,1972.0,NaN,513.0,565.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1630-N...,Beaches MLS,F10443703,N,Y,26.252622,-80.085693,https://www.redfin.com/FL/Pompano-Beach/1630-N...,Travis Paino,"Blue Realty Team, LLC",None,None,Walter Blackburn,Blacksmith Realty Group,None,None,6,blue
6,PAST SALE,2024-10-11,Condo/Co-op,1700 SE 15th St #111,Fort Lauderdale,FL,33316.0,500000.0,2.0,2.0,Caravelle,1200.0,NaN,1965.0,NaN,417.0,676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1700...,Beaches MLS,F10451896,N,Y,26.102984,-80.123814,https://www.redfin.com/FL/Fort-Lauderdale/1700...,Moe Mossa,"Savvy Avenue, LLC",None,None,Joseph Digirolamo,"Triangle Properties, Inc.",None,None,7,blue
7,PAST SALE,2024-10-08,Condo/Co-op,3000 E Sunrise Blvd Unit 6G,Fort Lauderdale,FL,33304.0,475000.0,2.0,2.0,Carlton Tower,1202.0,NaN,1969.0,NaN,395.0,869.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3000...,Beaches MLS,F10458957,N,Y,26.137635,-80.105026,https://www.redfin.com/FL/Fort-Lauderdale/3000...,Thomas Salomone,Real Estate II Inc,None,None,Howard Selaj,Coldwell Banker Realty,None,None,8,blue
8,PAST SALE,2024-10-11,Condo/Co-op,3200 Port Royale Dr N #1402,Fort Lauderdale,FL,33308.0,4499

In [114]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [115]:
m.save('index.html')

## Data snagger

In [116]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [117]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-07,Condo/Co-op,615 Bayshore Dr #303,Fort Lauderdale,FL,33304.0,1200000.0,3.0,3.0,LA Cascade Condo,2365.0,NaN,2003.0,NaN,507.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Beaches MLS,RX-11016889,N,Y,26.131947,-80.109184,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Lawrence Mastropieri,Mastropieri Group LLC,Joy Mastropieri,Mastropieri Group LLC,Robin Johannan,Fortune Christie's International Real Estate,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [118]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/BC_County_condo_sales_week_ending_10142024


# CREATE TEMPLATE 

In [119]:
muni_set = set(df_top_ten['CITY'])

In [120]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [121]:
df_top_ten.reset_index(inplace=True,drop=True)

In [122]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [123]:
top_sale

'LA Cascade Condo at 615 Bayshore Dr #303 in Fort Lauderdale'

In [124]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [125]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [126]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-07,Condo/Co-op,615 Bayshore Dr #303,Fort Lauderdale,FL,33304.0,1200000.0,3.0,3.0,LA Cascade Condo,2365.0,NaN,2003.0,NaN,507.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Beaches MLS,RX-11016889,N,Y,26.131947,-80.109184,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Lawrence Mastropieri,Mastropieri Group LLC,Joy Mastropieri,Mastropieri Group LLC,Robin Johannan,Fortune Christie's International Real Estate,None,None,1,orange,LA Cascade Condo at 615 Bayshore Dr #303 in Fo...
1,PAST SALE,2024-10-09,Condo/Co-op,401 N Birch Rd #1102,Fort Lauderdale,FL,33304.0,1200000.0,2.0,2.0,Tiffany House Residences,1286.0,NaN,2018.0,NaN,933.0,1888.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Beaches MLS,F10455561,N,Y,26.128334,-80.106186,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Dayana Susterman,"Compass Florida, LLC",Joshua Dotoli,"Compass Florida, LLC",Deborah Haines,Trophy Properties South Florida Inc,None,None,2,blue,Tiffany House Residences at 401 N Birch Rd #11...
2,PAST SALE,2024-10-11,Condo/Co-op,615 Bayshore Dr #401,Fort Lauderdale,FL,33304.0,1050000.0,2.0,2.5,LA Cascade Condo,2274.0,NaN,2003.0,NaN,462.0,1650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Beaches MLS,RX-10942553,N,Y,26.131947,-80.109184,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Brian Dratt,Partnership Realty Inc.,None,None,Brian Dratt,Partnership Realty Inc.,None,None,3,blue,LA Cascade Condo at 615 Bayshore Dr #401 in Fo...
3,PAST SALE,2024-10-07,Condo/Co-op,111 N Pompano Beach Blvd #1411,Pompano Beach,FL,33062.0,715000.0,2.0,2.0,Sea Monarch,1455.0,NaN,1970.0,NaN,491.0,1020.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/111-N-...,Beaches MLS,F10459447,N,Y,26.234032,-80.090070,https://www.redfin.com/FL/Pompano-Beach/111-N-...,Chad Bishop,One Sotheby's Int'l Realty,Michael Clouston,One Sotheby's Int'l Realty,Robin Michel Gervais,Best of Luxury Realty Corp.,None,None,4,blue,Sea Monarch at 111 N Pompano Beach Blvd #1411 ...
4,PAST SALE,2024-10-10,Condo/Co-op,110 N Federal Hwy #805,Fort Lauderdale,FL,33301.0,520000.0,2.0,2.0,The Waverly At Las Olas,1350.0,1350.0,2004.0,NaN,385.0,1389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/110-...,Beaches MLS,F10450734,N,Y,26.123305,-80.136965,https://www.redfin.com/FL/Fort-Lauderdale/110-...,Adam Curtis,LoKation,None,None,Adam Curtis,LoKation,None,None,5,blue,The Waverly At Las Olas at 110 N Federal Hwy #...
5,PAST SALE,2024-10-07,Condo/Co-op,1630 N Ocean Blvd #712,Pompano Beach,FL,33062.0,500000.0,2.0,2.0,Tiffany Gardens West,975.0,NaN,1972.0,NaN,513.0,565.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1630-N...,Beaches MLS,F10443703,N,Y,26.252622,-80.085693,https://www.redfin.com/FL/Pompano-Beach/1630-N...,Travis Paino,"Blue Realty Team, LLC",None,None,Walter Blackburn,Blacksmith Realty Group,None,None,6,blue,Tiffany Gardens West at 1630 N Ocean Blvd #712...
6,PAST SALE,2024-10-11,Condo/Co-op,1700 SE 15th St #111,Fort Lauderdale,FL,33316.0,500000.0,2.0,2.0,Caravelle,1200.0,NaN,1965.0,NaN,417.0,676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1700...,Beaches MLS,F10451896,N,Y,26.102984,-80.123814,https://www.redfin.com/FL/Fort-Lauderdale/1700...,Moe Mossa,"Savvy Avenue, LLC",None,None,Joseph Digirolamo,"Triangle Properties, Inc.",None,None,7,blue,Caravelle at 1700 SE 15th St #111 in Fort Laud...
7,PAST SALE,2024-10-08,Condo/Co-op,3000 E Sunrise Blvd Unit 6G,Fort Lauderdale,FL,33304.0,475000.0,2.0,2.0,Carlton Tower,1202.0,NaN,1969.0,NaN,395.

In [127]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: LA Cascade Condo closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $430,000M to $1,200,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Pompano Beach, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 70 condo sales totaling $20,625,400 million from Oct. 8th to Oct. 15th. The previous week, brokers closed 142 condo sales totaling $55,100,000.

Last week’s units sold for an average of $294,649, lower than the $388,150 average price from the previous week. The average price per square foot fell to $253 from $305, according t

In [128]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [129]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [130]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [131]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-07,Condo/Co-op,615 Bayshore Dr #303,Fort Lauderdale,FL,33304.0,1200000.0,3.0,3.0,LA Cascade Condo,2365.0,NaN,2003.0,NaN,507.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Beaches MLS,RX-11016889,N,Y,26.131947,-80.109184,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Lawrence Mastropieri,Mastropieri Group LLC,Joy Mastropieri,Mastropieri Group LLC,Robin Johannan,Fortune Christie's International Real Estate,None,None,1,orange,LA Cascade Condo at 615 Bayshore Dr #303 in Fo...
1,PAST SALE,2024-10-09,Condo/Co-op,401 N Birch Rd #1102,Fort Lauderdale,FL,33304.0,1200000.0,2.0,2.0,Tiffany House Residences,1286.0,NaN,2018.0,NaN,933.0,1888.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Beaches MLS,F10455561,N,Y,26.128334,-80.106186,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Dayana Susterman,"Compass Florida, LLC",Joshua Dotoli,"Compass Florida, LLC",Deborah Haines,Trophy Properties South Florida Inc,None,None,2,blue,Tiffany House Residences at 401 N Birch Rd #11...
2,PAST SALE,2024-10-11,Condo/Co-op,615 Bayshore Dr #401,Fort Lauderdale,FL,33304.0,1050000.0,2.0,2.5,LA Cascade Condo,2274.0,NaN,2003.0,NaN,462.0,1650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Beaches MLS,RX-10942553,N,Y,26.131947,-80.109184,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Brian Dratt,Partnership Realty Inc.,None,None,Brian Dratt,Partnership Realty Inc.,None,None,3,blue,LA Cascade Condo at 615 Bayshore Dr #401 in Fo...
3,PAST SALE,2024-10-07,Condo/Co-op,111 N Pompano Beach Blvd #1411,Pompano Beach,FL,33062.0,715000.0,2.0,2.0,Sea Monarch,1455.0,NaN,1970.0,NaN,491.0,1020.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/111-N-...,Beaches MLS,F10459447,N,Y,26.234032,-80.090070,https://www.redfin.com/FL/Pompano-Beach/111-N-...,Chad Bishop,One Sotheby's Int'l Realty,Michael Clouston,One Sotheby's Int'l Realty,Robin Michel Gervais,Best of Luxury Realty Corp.,None,None,4,blue,Sea Monarch at 111 N Pompano Beach Blvd #1411 ...
4,PAST SALE,2024-10-10,Condo/Co-op,110 N Federal Hwy #805,Fort Lauderdale,FL,33301.0,520000.0,2.0,2.0,The Waverly At Las Olas,1350.0,1350.0,2004.0,NaN,385.0,1389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/110-...,Beaches MLS,F10450734,N,Y,26.123305,-80.136965,https://www.redfin.com/FL/Fort-Lauderdale/110-...,Adam Curtis,LoKation,None,None,Adam Curtis,LoKation,None,None,5,blue,The Waverly At Las Olas at 110 N Federal Hwy #...
5,PAST SALE,2024-10-07,Condo/Co-op,1630 N Ocean Blvd #712,Pompano Beach,FL,33062.0,500000.0,2.0,2.0,Tiffany Gardens West,975.0,NaN,1972.0,NaN,513.0,565.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1630-N...,Beaches MLS,F10443703,N,Y,26.252622,-80.085693,https://www.redfin.com/FL/Pompano-Beach/1630-N...,Travis Paino,"Blue Realty Team, LLC",None,None,Walter Blackburn,Blacksmith Realty Group,None,None,6,blue,Tiffany Gardens West at 1630 N Ocean Blvd #712...
6,PAST SALE,2024-10-11,Condo/Co-op,1700 SE 15th St #111,Fort Lauderdale,FL,33316.0,500000.0,2.0,2.0,Caravelle,1200.0,NaN,1965.0,NaN,417.0,676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1700...,Beaches MLS,F10451896,N,Y,26.102984,-80.123814,https://www.redfin.com/FL/Fort-Lauderdale/1700...,Moe Mossa,"Savvy Avenue, LLC",None,None,Joseph Digirolamo,"Triangle Properties, Inc.",None,None,7,blue,Caravelle at 1700 SE 15th St #111 in Fort Laud...
7,PAST SALE,2024-10-08,Condo/Co-op,3000 E Sunrise Blvd Unit 6G,Fort Lauderdale,FL,33304.0,475000.0,2.0,2.0,Carlton Tower,1202.0,NaN,1969.0,NaN,395.

In [140]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: LA Cascade Condo closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $430,000M to $1,200,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Pompano Beach, Fort Lauderdale

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 70 condo sales totaling $20,625,400 million from Oct. 8th to Oct. 15th. The previous week, brokers closed 142 condo sales totaling $55,100,000.

Last week’s units sold for an average of $294,649, lower than the $388,150 average price from the previous week. The average price per square foot fell to $253 from $305, according t

In [141]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Fort-Lauderdale/615-Bayshore-Dr-33304/unit-303/home/42019740


In [142]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/401-N-Birch-Rd-33304/unit-1102/home/145953781


In [143]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Pompano-Beach/1500-S-Ocean-Blvd-33062/unit-1007/home/41702223


In [144]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Fort-Lauderdale/3200-Port-Royale-Dr-N-33308/unit-1402/home/41703655


In [145]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/401-N-Birch-Rd-33304/unit-1102/home/145953781


In [146]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Pompano-Beach/1500-S-Ocean-Blvd-33062/unit-1007/home/41702223


In [147]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-07,Condo/Co-op,615 Bayshore Dr #303,Fort Lauderdale,FL,33304.0,1200000.0,3.0,3.0,LA Cascade Condo,2365.0,NaN,2003.0,NaN,507.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Beaches MLS,RX-11016889,N,Y,26.131947,-80.109184,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Lawrence Mastropieri,Mastropieri Group LLC,Joy Mastropieri,Mastropieri Group LLC,Robin Johannan,Fortune Christie's International Real Estate,None,None,1,orange,LA Cascade Condo at 615 Bayshore Dr #303 in Fo...
1,PAST SALE,2024-10-09,Condo/Co-op,401 N Birch Rd #1102,Fort Lauderdale,FL,33304.0,1200000.0,2.0,2.0,Tiffany House Residences,1286.0,NaN,2018.0,NaN,933.0,1888.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Beaches MLS,F10455561,N,Y,26.128334,-80.106186,https://www.redfin.com/FL/Fort-Lauderdale/401-...,Dayana Susterman,"Compass Florida, LLC",Joshua Dotoli,"Compass Florida, LLC",Deborah Haines,Trophy Properties South Florida Inc,None,None,2,blue,Tiffany House Residences at 401 N Birch Rd #11...
2,PAST SALE,2024-10-11,Condo/Co-op,615 Bayshore Dr #401,Fort Lauderdale,FL,33304.0,1050000.0,2.0,2.5,LA Cascade Condo,2274.0,NaN,2003.0,NaN,462.0,1650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Beaches MLS,RX-10942553,N,Y,26.131947,-80.109184,https://www.redfin.com/FL/Fort-Lauderdale/615-...,Brian Dratt,Partnership Realty Inc.,None,None,Brian Dratt,Partnership Realty Inc.,None,None,3,blue,LA Cascade Condo at 615 Bayshore Dr #401 in Fo...
3,PAST SALE,2024-10-07,Condo/Co-op,111 N Pompano Beach Blvd #1411,Pompano Beach,FL,33062.0,715000.0,2.0,2.0,Sea Monarch,1455.0,NaN,1970.0,NaN,491.0,1020.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/111-N-...,Beaches MLS,F10459447,N,Y,26.234032,-80.090070,https://www.redfin.com/FL/Pompano-Beach/111-N-...,Chad Bishop,One Sotheby's Int'l Realty,Michael Clouston,One Sotheby's Int'l Realty,Robin Michel Gervais,Best of Luxury Realty Corp.,None,None,4,blue,Sea Monarch at 111 N Pompano Beach Blvd #1411 ...
4,PAST SALE,2024-10-10,Condo/Co-op,110 N Federal Hwy #805,Fort Lauderdale,FL,33301.0,520000.0,2.0,2.0,The Waverly At Las Olas,1350.0,1350.0,2004.0,NaN,385.0,1389.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/110-...,Beaches MLS,F10450734,N,Y,26.123305,-80.136965,https://www.redfin.com/FL/Fort-Lauderdale/110-...,Adam Curtis,LoKation,None,None,Adam Curtis,LoKation,None,None,5,blue,The Waverly At Las Olas at 110 N Federal Hwy #...
5,PAST SALE,2024-10-07,Condo/Co-op,1630 N Ocean Blvd #712,Pompano Beach,FL,33062.0,500000.0,2.0,2.0,Tiffany Gardens West,975.0,NaN,1972.0,NaN,513.0,565.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1630-N...,Beaches MLS,F10443703,N,Y,26.252622,-80.085693,https://www.redfin.com/FL/Pompano-Beach/1630-N...,Travis Paino,"Blue Realty Team, LLC",None,None,Walter Blackburn,Blacksmith Realty Group,None,None,6,blue,Tiffany Gardens West at 1630 N Ocean Blvd #712...
6,PAST SALE,2024-10-11,Condo/Co-op,1700 SE 15th St #111,Fort Lauderdale,FL,33316.0,500000.0,2.0,2.0,Caravelle,1200.0,NaN,1965.0,NaN,417.0,676.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1700...,Beaches MLS,F10451896,N,Y,26.102984,-80.123814,https://www.redfin.com/FL/Fort-Lauderdale/1700...,Moe Mossa,"Savvy Avenue, LLC",None,None,Joseph Digirolamo,"Triangle Properties, Inc.",None,None,7,blue,Caravelle at 1700 SE 15th St #111 in Fort Laud...
7,PAST SALE,2024-10-08,Condo/Co-op,3000 E Sunrise Blvd Unit 6G,Fort Lauderdale,FL,33304.0,475000.0,2.0,2.0,Carlton Tower,1202.0,NaN,1969.0,NaN,395.

## Time on Market Calculator

In [77]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 9) ## List (Earlier) date
date2 = datetime(2024, 10, 9) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

61


## Clean CSV for Datawrapper Chart

In [148]:
chart_df = df_top_ten

In [149]:
chart_df = chart_df.fillna(" ")

In [150]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [151]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [152]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [153]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [154]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [155]:
csv_date_string = today.strftime("%m_%d_%Y")

In [156]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")